In [85]:
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from matplotlib import pyplot
from gensim.models import KeyedVectors
import pandas as pd
import numpy as np
import time
import random
from numpy.testing import assert_almost_equal

In [14]:
import urllib.request
import os

def download_file(url, local_file):
    """
    Helper function to download a file and store it locally
    """
    if not os.path.exists(local_file):
        print('Downloading')
        with urllib.request.urlopen(url) as opener, \
             open(local_file, mode='wb') as outfile:
                    outfile.write(opener.read())
    else:
        print('Already downloaded')

In [15]:
google_w2v_file = 'data/GoogleNews-vectors-negative300.bin'
download_file('https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz',
              google_w2v_file+'.gz')

Downloading


In [16]:
!if [ ! -f data/GoogleNews-vectors-negative300.bin ]; then gunzip data/GoogleNews-vectors-negative300.bin.gz; fi

In [17]:
news_model = KeyedVectors.load_word2vec_format(google_w2v_file, binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [18]:
df = pd.read_csv('edges.csv', header=None)

In [19]:
np.random.seed(100)
vectors = {}
unknown_vector = np.random.rand(300)

Checking different link predictions

In [109]:
# link 1
array_0 = -0.017254869469058274 #Peanut butter cookie
array_1 = -0.012768545150756837 #Cookie

In [107]:
# link 1
array_0 = 0.009587866465250651 #PricewaterhouseCoopers
array_1 = -0.018883171081542968 #Pakistan

In [112]:
# link 1
array_0 = -0.013354159493367964 #Internet service provider
array_1 = -0.0011646270751953125 #Camcorder

In [114]:
# link 0
array_0 = -0.007197036743164063 #Makita
array_1 = 0.008356563250223796 #Stranger Things

In [116]:
# link 0
array_0 = -0.011751492818196615#Spartanburg
array_1 = -0.031591059366861976 #iPhone

In [90]:
def check(a, b):
    try:
        assert_almost_equal(a, b)
        return True
    except:
        return False

In [104]:
def look_up(input_array_mean):
    for name, array in vectors.items():  # Searching in dictionary to find specific array
        if check(array.mean(), input_array_mean):
            print(name)

In [117]:
look_up(array_0)
look_up(array_1)

Spartanburg
iPhone


In [20]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [21]:
def create_vector(input):
    avg_vec=[]
    for i in input.split():
        try:
            avg_vec.append(news_model.word_vec(i))
        except:
            # unk value assigned to unk words
            avg_vec.append(unknown_vector)
    return np.mean(avg_vec, axis=0)

In [22]:
start_time = time.time()
new_df = pd.DataFrame(columns=["first", "second", "link"])
for i in range(df.shape[0]):
    #print(str(i)+" ", end='')
    first = df.loc[i][0]
    second = df.loc[i][1]
    # check if vector already in dictionary
    if first in vectors:
        #if yes, insert in new df
        new_df.loc[i, 'first'] = vectors[first]
    else:
        try:
            vector = news_model.word_vec(first.replace(" ", "_"))
        except:
            vector = create_vector(first)
        #insert vector in dictionary
        vectors[first] = vector
        new_df.loc[i, 'first'] = vector
    # do the same 4 second 
    if second in vectors:
        new_df.loc[i, 'second'] = vectors[second]
    else:
        try:
            vector = news_model.word_vec(second.replace(" ", "_"))
        except:
            vector = create_vector(second)
        vectors[second] = vector
        new_df.loc[i, 'second'] = vector
    new_df.loc[i, 'link'] = 1

end_time = time.time()
epoch_mins, epoch_secs = epoch_time(start_time, end_time)
print(f'\nTime: {epoch_mins}m {epoch_secs}s')


Time: 4m 19s


In [ ]:
temp = df[df[0] == "All-star game"] 
temp
#temp[temp[1] == 'Tefdgfdgdst'].shape[0]

In [ ]:
start_time = time.time()
df_0 = pd.DataFrame(columns=["first", "second", "link"])
for i, v in enumerate(range(new_df.shape[0])):
    #print(str(i)+" ", end='')
    #we select a random value to choose from df
    random_num = random.randint(0, df.shape[0]-1)
    #we take the random first vector
    first = df.iloc[random_num][0]
    random_num = random.randint(0, df.shape[0]-1)
    second = df.iloc[random_num][1]

    #we check if the two words exist in original df
    temp = df[df[0] == first] 
    condition = temp[temp[1] == second].shape[0] 
    if not condition:
        df_0.loc[i] = [first, second, 0]
end_time = time.time()
epoch_mins, epoch_secs = epoch_time(start_time, end_time)
print(f'\nTime: {epoch_mins}m {epoch_secs}s')


Time: 18m 10s


In [ ]:
# resetting id for problem of link 1
df_0.reset_index(inplace=True, drop=True)

In [ ]:
start_time = time.time()
vec_df_0 = pd.DataFrame(columns=["first", "second", "link"])
for i in range(df_0.shape[0]):
    print(str(i)+" ", end='')
    first = df_0.loc[i][0]
    second = df_0.loc[i][1]
    # check if vector already in dictionary
    if first in vectors:
        #if yes, insert in new df
        vec_df_0.loc[i, 'first'] = vectors[first]
    else:
        try:
            vector = news_model.word_vec(first.replace(" ", "_"))
        except:
            vector = create_vector(first)
        #insert vector in dictionary
        vectors[first] = vector
        vec_df_0.loc[i, 'first'] = vector
    # do the same 4 second 
    if second in vectors:
        vec_df_0.loc[i, 'second'] = vectors[second]
    else:
        try:
            vector = news_model.word_vec(second.replace(" ", "_"))
        except:
            vector = create_vector(second)
        vectors[second] = vector
        vec_df_0.loc[i, 'second'] = vector
    vec_df_0.loc[i, 'link'] = 0

end_time = time.time()
epoch_mins, epoch_secs = epoch_time(start_time, end_time)
print(f'\nTime: {epoch_mins}m {epoch_secs}s')

In [ ]:
#Concatanating two dataframes link 1 and 0
result = pd.concat([new_df, vec_df_0])

In [ ]:
result['link'].value_counts()

1    70646
0    70469
Name: link, dtype: int64

In [ ]:
result.reset_index(inplace=True, drop=True)

In [ ]:
result.to_pickle("embedding_pickle.csv")

In [ ]:
embedding_df_pickle = pd.read_pickle('embedding_pickle.csv')
embedding_df_pickle

,first,second,link
0,"[0.078552246, 0.1998291, 0.11755371, 0.1346435...","[-0.1375, 0.10991211, 0.13271484, 0.05996094, ...",1
1,"[0.078552246, 0.1998291, 0.11755371, 0.1346435...","[-0.19732666, 0.035949707, 0.16992188, 0.08459...",1
2,"[0.078552246, 0.1998291, 0.11755371, 0.1346435...","[-0.265625, -0.16503906, 0.12402344, 0.1982421...",1
3,"[0.078552246, 0.1998291, 0.11755371, 0.1346435...","[-0.099121094, 0.07373047, 0.0625, -0.12207031...",1
4,"[0.078552246, 0.1998291, 0.11755371, 0.1346435...","[0.5434049417909654, 0.27836938509379616, 0.42...",1
...,...,...,...
141110,"[0.09033203, 0.123046875, -0.20605469, -0.3203...","[0.100097656, 0.19042969, 0.10986328, 0.384765...",0
141111,"[-0.006286621, -0.09326172, 0.33398438, -0.083...","[0.125, 0.15039062, -0.1953125, -0.24707031, 0...",0
141112,"[0.13964844, 0.20703125, -0.111816406, 0.25585...","[0.115722656, -0.111816406, -0.30078125, 0.066...",0
141113,"[0.14497599130711636, 0.17866217049219904, 0.0...","[-0.15332031, 0.099121094, -0.02722168, 0.1796...",0


In [122]:
news_model.word_vec("Italy")

array([ 5.78613281e-02,  1.36718750e-01,  1.69921875e-01,  7.27539062e-02,
       -1.51367188e-01, -1.47247314e-03, -1.51367188e-01, -3.12500000e-01,
       -7.12890625e-02,  2.61718750e-01,  2.33650208e-04, -6.07910156e-02,
        9.52148438e-02, -1.12792969e-01, -5.81054688e-02,  4.58984375e-02,
        1.89208984e-02, -2.39257812e-02,  4.98046875e-02,  1.61132812e-01,
        1.25000000e-01,  1.63574219e-02,  1.85546875e-01, -6.49414062e-02,
        2.61230469e-02,  5.41992188e-02, -5.22460938e-02,  3.36914062e-02,
       -2.11914062e-01,  1.04492188e-01,  9.22851562e-02, -8.83789062e-02,
       -2.67028809e-03,  1.14746094e-01, -4.19921875e-02, -2.38281250e-01,
       -2.75390625e-01,  4.24804688e-02, -4.32128906e-02,  6.83593750e-02,
        6.39648438e-02,  1.58203125e-01,  1.59179688e-01,  4.02832031e-02,
       -3.20434570e-03,  1.81640625e-01, -1.86767578e-02,  3.59375000e-01,
       -6.73828125e-02,  1.67968750e-01, -1.98242188e-01,  3.39843750e-01,
        1.57470703e-02,  